<a href="https://colab.research.google.com/github/AmirJlr/NLP-FUM/blob/master/Hands%20On/perisan_tweets_fast_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tweet-preprocessor
!pip install pandas
!pip install numpy
!pip install pytorch
!pip install hazm
!pip install sentence_transformers
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: Could not build wheels for pytorch, which is required to install pyproject.toml-based projects
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━

In [2]:
import re
import csv
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch

import preprocessor as p
from hazm import *
from sentence_transformers.util import community_detection
from transformers import AutoConfig, AutoTokenizer, AutoModel


In [6]:
url ='https://raw.githubusercontent.com/AmirJlr/NLP-FUM/master/Hands%20On/tweets.csv'

tweets = pd.read_csv(url)

<ipython-input-6-791ecbed8fa3>:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets = pd.read_csv(url)


In [9]:
tweets['tweet'][0]

'نسل جوان ما بحمدالله الگوهای درخشان و برجسته و کم\u200cنظیری را هم میشناسد:\n\nاز شهید چمران و شهید آوینی تا شهدای هسته\u200cای و تا #شهید_سلیمانی و تا شهید بزرگوار مطهری.\n\nدرود خدا بر شهیدان و سپاس ما به معلمان و خوشامد ما نثار نسل جوان خوش عاقبت ملت ایران باد./۶\n\n#روز_معلم'

In [10]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84210 entries, 0 to 84209
Data columns (total 34 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               84210 non-null  int64  
 1   conversation_id  84210 non-null  int64  
 2   created_at       84210 non-null  int64  
 3   date             84210 non-null  object 
 4   time             84210 non-null  object 
 5   timezone         84210 non-null  object 
 6   user_id          84210 non-null  int64  
 7   username         84210 non-null  object 
 8   name             84210 non-null  object 
 9   place            42 non-null     object 
 10  tweet            84210 non-null  object 
 11  mentions         84210 non-null  object 
 12  urls             84210 non-null  object 
 13  photos           84210 non-null  object 
 14  replies_count    84210 non-null  int64  
 15  retweets_count   84210 non-null  int64  
 16  likes_count      84210 non-null  int64  
 17  hashtags    

In [11]:
tweets = tweets['tweet']

In [12]:
tweets = pd.DataFrame(tweets)

In [13]:
tweets

,tweet
0,نسل جوان ما بحمدالله الگوهای درخشان و برجسته و...
1,نسل جوانی که در مسیر #علم_نافع، برای پیشرفت کش...
2,کودک و نوجوان و جوان در کشور اسلامی می‌آموزد ک...
3,در مدرسه نبوت،نسلهای بشر با کتاب وحکمت،آموزش و...
4,این سخن امام خمینی که معلمی را شغل انبیا دانست...
...,...
84205,این گفتگو در جمع دوستان بدرقه کننده و پیش از ع...
84206,نقش #احمدی_نژاد در تعطیل شدن دانشگاه‌ها به بها...
84207,در اهمیّت حرکت به سوی #شایسته_سالاری https://t...
84208,تأکید 9چهره اصلاحات بر شایسته سالاری\nنامه به ...


In [15]:
normalizer = Normalizer()


emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emotions
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)


characters = re.compile("""[A-Za-z0-9~`!#$%\^&*+=\-\[\]\\;,)(/{}|\\:<>\?«»"']+""")


def cleanTweet(tweet):
    p.set_options(p.OPT.URL, p.OPT.MENTION)
    tweet = p.clean(tweet).strip() #remove URLs and Mentions
    tweet = re.sub(characters, '', tweet)
    tweet = re.sub(emoji_pattern,'',tweet)
    tweet = normalizer.normalize(tweet)
    if len(tweet) > 10:
      return tweet



In [16]:
tweets['tweet'] = tweets['tweet'].apply(cleanTweet)

In [17]:
tweets = tweets[tweets['tweet'].astype(bool)].dropna().sample(frac=1).reset_index(drop=True)

In [25]:
tweets

,tweet
0,شرح جنایت …
1,شاید بتوان تفسیر دیگری داشت، مثل جامعه‌ی اغلب ...
2,افتخار نداریم، معمولا یه سر میام، ایونت‌های شم...
3,شاید هم بشه کاری کرد
4,پیشنهاد من اینه که اگر رانندگی میکنید موقع ران...
...,...
75462,سرم درد میکنه
75463,در بحث کودک‌همسری در مجلس در مراحل پایانی کار ...
75464,کشورها بحران کنونی را فرصتی برای اعمال تغییرات...
75465,حاصل مباحثاتی سخت در کمیسیون دسترسی آزاد به اط...


In [19]:
config = AutoConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model = AutoModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")


Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
input_ids = torch.tensor(tokenizer.encode(tweets['tweet'][0])).unsqueeze(0)
input_ids

tensor([[   2, 5065, 6527, 1034,    4]])

In [21]:
outputs = model(input_ids)
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.4886,  0.0342,  0.6719,  ...,  0.4595,  1.6636, -1.3608],
         [-0.1801, -0.0729,  0.6122,  ...,  1.1971,  1.6862, -2.1530],
         [-0.0879,  0.2367,  0.3135,  ...,  0.4613,  0.6223, -0.2007],
         [-0.1998,  0.2966, -0.1828,  ...,  0.2479,  0.7304, -1.5701],
         [ 0.0969, -0.1106,  0.4700,  ...,  0.0423,  0.1049, -0.0881]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 7.9659e-02,  9.9794e-01, -1.9861e-01, -9.7559e-01,  4.5639e-01,
          3.0776e-01,  2.2927e-01, -7.8813e-02, -9.7272e-01,  1.5453e-01,
         -2.1409e-01,  6.0008e-01,  1.6396e-01,  9.8019e-01, -3.7414e-01,
          8.5836e-01, -3.7387e-01, -2.5922e-01, -7.3599e-02,  6.1472e-02,
          4.1646e-01,  5.8268e-01,  3.0346e-01,  4.7501e-01,  4.7252e-01,
         -8.4503e-02,  5.5198e-02,  7.8013e-02, -4.8463e-01, -6.2037e-01,
          1.4482e-01, -4.9138e-02,  9.9974e-01,  6.4569e-01,  3.1709e-01,
    

In [22]:
outputs = outputs['pooler_output']
outputs

tensor([[ 7.9659e-02,  9.9794e-01, -1.9861e-01, -9.7559e-01,  4.5639e-01,
          3.0776e-01,  2.2927e-01, -7.8813e-02, -9.7272e-01,  1.5453e-01,
         -2.1409e-01,  6.0008e-01,  1.6396e-01,  9.8019e-01, -3.7414e-01,
          8.5836e-01, -3.7387e-01, -2.5922e-01, -7.3599e-02,  6.1472e-02,
          4.1646e-01,  5.8268e-01,  3.0346e-01,  4.7501e-01,  4.7252e-01,
         -8.4503e-02,  5.5198e-02,  7.8013e-02, -4.8463e-01, -6.2037e-01,
          1.4482e-01, -4.9138e-02,  9.9974e-01,  6.4569e-01,  3.1709e-01,
         -9.2210e-01,  1.7977e-02, -6.3189e-01, -8.6811e-01, -9.0906e-01,
          7.6531e-01, -6.5704e-01,  9.9414e-01,  3.1085e-01,  7.0151e-01,
         -9.9861e-01, -4.3315e-01,  5.8681e-01, -1.3455e-01, -8.1893e-02,
          2.4027e-01, -8.1794e-02,  1.0887e-01, -8.1938e-01,  1.4510e-02,
          9.9756e-01, -5.4217e-01, -2.2210e-01, -5.8024e-01,  1.6472e-01,
         -2.7866e-02,  3.8016e-01,  3.2239e-01, -2.2500e-01, -4.9459e-01,
         -3.2188e-01, -2.5939e-01,  8.

In [23]:
outputs.shape

torch.Size([1, 768])

In [24]:

with open('embeddings','wb') as file:
  batch_size = 50
  for batch in tqdm(range(int(len(tweets['tweet'])/batch_size)+1)):
    embeddings = []
    start = batch*batch_size
    end = (batch+1)*batch_size
    if end > len(tweets['tweet']):
      end = len(tweets['tweet'])
    for tweet in tweets['tweet'][start:end]:
      input_ids = torch.tensor(tokenizer.encode(tweet)).unsqueeze(0)
      outputs = model(input_ids)
      embeddings.append(outputs['pooler_output'].detach().numpy().squeeze())
    np.savetxt(file, embeddings)


100%|██████████| 1510/1510 [2:47:08<00:00,  6.64s/it]


In [26]:
embeddings = np.loadtxt('embeddings')
len(embeddings)

75467

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

clusters = community_detection(embeddings, threshold=0.9,) #, init_max_size=len(embeddings)

AttributeError: ignored

In [ ]:
def write_to_csv_file(rows, OUTPUT_PATH):
    with open(OUTPUT_PATH, mode="w") as fp:
        fieldnames = [
            "cluster_id", 
            "tweet"]
        writer = csv.DictWriter(fp, fieldnames=fieldnames)
        writer.writeheader()
        for row in rows:
            writer.writerow(row)

In [ ]:
def collect_data(ds, clusters):
    rows = []
    for i, tweet_ids in enumerate(clusters):
        cluster_rows = []
        for tweet_id in tweet_ids:
            cluster_rows.append(
                {
                    "cluster_id": str(i + 1),
                    "tweet": ds["tweet"][tweet_id],
                }
            )
        cluster_rows = map(dict, set(tuple(sorted(d.items())) for d in cluster_rows))
        rows.extend(cluster_rows)
    return rows

In [ ]:
rows = collect_data(tweets, clusters)

In [ ]:
write_to_csv_file(rows, 'tweets_clustered.csv')

In [ ]:
clusters = pd.read_csv("tweets_clustered.csv")

for cluster_id in clusters['cluster_id'].unique():
    print(f"Cluster-{cluster_id}")
    for tweet in clusters[clusters['cluster_id']==cluster_id]['tweet'][:10]:
        print(tweet)
    print("\n")